In [1]:
import torch.nn as nn
import torch
import matplotlib.pyplot as plt 
import pandas as pd

from sklearn.datasets import load_iris
import numpy as np

In [2]:
torch.manual_seed(2)

# Load Iris Dataset

In [3]:
iris = load_iris()
X = iris.data
Y = iris.target

Use train_test_split to split data into train and validation set

In [4]:
from sklearn.model_selection import train_test_split
x, x_val, y, y_val = train_test_split(X, Y, test_size=0.33, random_state=42)

In [5]:
x.shape, y.shape, x_val.shape, y_val.shape

((100, 4), (100,), (50, 4), (50,))

In [6]:
x.shape

(100, 4)

In [7]:
x_train = x.reshape(-1, x.shape[1]).astype('float32')
y_train = y

x_val = x_val.reshape(-1, x_val.shape[1]).astype('float32')
y_val = y_val

Define validation data as a Pytorch Tensor

In [8]:
x_val = torch.from_numpy(x_val)
y_val = torch.from_numpy(y_val)

Put Data through DataLoader, so we can use batches

In [9]:
from torch.utils.data import Dataset, DataLoader
class Data(Dataset):
    def __init__(self):
        self.x=torch.from_numpy(x_train)
        self.y=torch.from_numpy(y_train)
        self.len=self.x.shape[0]
    def __getitem__(self,index):      
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

In [10]:
data_set=Data()

In [11]:
trainloader=DataLoader(dataset=data_set,batch_size=64)

In [12]:
data_set.x[1:10]

tensor([[7.6000, 3.0000, 6.6000, 2.1000],
        [5.6000, 3.0000, 4.5000, 1.5000],
        [5.1000, 3.5000, 1.4000, 0.2000],
        [7.7000, 2.8000, 6.7000, 2.0000],
        [5.8000, 2.7000, 4.1000, 1.0000],
        [5.2000, 3.4000, 1.4000, 0.2000],
        [5.0000, 3.5000, 1.3000, 0.3000],
        [5.1000, 3.8000, 1.9000, 0.4000],
        [5.0000, 2.0000, 3.5000, 1.0000]])

In [13]:
data_set.y[1:10]

tensor([2, 1, 0, 2, 1, 0, 0, 0, 1])

In [14]:
data_set.x.shape, data_set.y.shape

(torch.Size([100, 4]), torch.Size([100]))

# Build Model and train it

Build softmax Classifier

In [15]:
class Net(nn.Module):
    def __init__(self,D_in,H,D_out):
        super(Net,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.linear2=nn.Linear(H,D_out)

        
    def forward(self,x):
        x=torch.sigmoid(self.linear1(x))  
        x=self.linear2(x)
        return x

In [16]:
input_dim=4     # how many Variables are in the dataset
hidden_dim = 25 # hidden layers
output_dim=3    # number of classes
input_dim

4

In [17]:
# Instantiate model
model=Net(input_dim,hidden_dim,output_dim)

View the size of the model parameters:

In [18]:
print('W:',list(model.parameters())[0].size())
print('b',list(model.parameters())[1].size())

W: torch.Size([25, 4])
b torch.Size([25])


Loss function

In [19]:
criterion=nn.CrossEntropyLoss()

Optimizer

In [20]:
learning_rate=0.1
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

In [21]:
n_epochs=1000
loss_list=[]

#n_epochs
for epoch in range(n_epochs):
    for x, y in trainloader:
      

        #clear gradient 
        optimizer.zero_grad()
        #make a prediction 
        z=model(x)
        # calculate loss, da Cross Entropy benutzt wird muss ich in den loss Klassen vorhersagen, 
        # also Wahrscheinlichkeit pro Klasse. Das mach torch.max(y,1)[1])
        loss=criterion(z,y)
        # calculate gradients of parameters 
        loss.backward()
        # update parameters 
        optimizer.step()
        
        loss_list.append(loss.data)
        
        
        #print('epoch {}, loss {}'.format(epoch, loss.item()))

In [22]:
z

tensor([[  7.9463,   2.9589, -11.4269],
        [ -5.9383,   1.7959,   3.4491],
        [ -2.0020,   3.1438,  -1.9223],
        [ -5.8968,   1.7673,   3.4422],
        [  0.4821,   3.8170,  -5.1312],
        [ -2.5616,   2.8210,  -1.0162],
        [ -2.8092,   2.9822,  -0.9587],
        [  6.5563,   3.2080, -10.4113],
        [ -1.9753,   3.4324,  -2.2979],
        [ -3.5453,   2.6669,   0.1052],
        [  8.0017,   2.9629, -11.4862],
        [ -1.4998,   3.2447,  -2.5265],
        [ -7.2485,   1.3297,   5.2447],
        [ -6.5980,   1.4648,   4.4796],
        [  7.8626,   2.9580, -11.3463],
        [ -0.3824,   3.4341,  -3.8589],
        [ -7.3588,   1.1140,   5.6299],
        [ -5.5482,   2.0996,   2.7045],
        [  8.0067,   2.9501, -11.4848],
        [ -7.2798,   1.1729,   5.4780],
        [  7.4811,   3.0565, -11.1110],
        [ -2.5956,   3.2327,  -1.4602],
        [ -7.0787,   1.4179,   4.9750],
        [ -6.8759,   1.3702,   4.8466],
        [ -1.8931,   3.2035,  -2.1071],


Each row represents a row from the original dataset. Each column represents a class. The first column represents the class 0, the second column class 1 and the third column class 2. The highest value for each row represents which class the model would put each row. For instance, the highest value in the first row is 9.3748, hence the predicted class is 0.  

Check model on Validation Set

In [23]:
z=model(x_val)

In [24]:
yhat=torch.max(z.data,1)
yhat

torch.return_types.max(
values=tensor([2.8089, 7.8910, 7.3815, 2.6484, 3.0109, 7.6734, 3.4687, 3.4220, 2.2595,
        3.3898, 2.8713, 7.5114, 8.0576, 7.5052, 7.9645, 2.8279, 6.1955, 3.2472,
        2.5801, 6.1737, 7.3503, 2.7248, 7.4677, 5.9607, 3.3796, 4.4848, 5.8301,
        5.9697, 7.3574, 7.2710, 8.1972, 8.3064, 3.5513, 7.5657, 7.6446, 4.4157,
        3.1946, 7.8349, 7.9584, 8.2255, 5.1776, 2.8847, 3.1414, 8.1717, 8.0019,
        3.4166, 2.4844, 4.2140, 3.4338, 5.7246]),
indices=tensor([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2,
        2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2,
        1, 2]))

In [25]:
y_val

tensor([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2,
        2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2,
        1, 2])

The full code can be found here: https://github.com/lschmiddey/PyTorch-Multiclass-Classification/blob/master/Softmax_Regression_Deep_Learning_Iris_Dataset.ipynb